# Reproducing PII Detection Results from Paper

This notebook reproduces the results from the paper "Detecting Personally Identifiable Information Through Natural Language Processing: A Step Forward" (asi-08-00055.pdf).

## Methodology Summary

- **Model**: BERT (Bidirectional Encoder Representations from Transformers)
- **Training Dataset**:
  - 43,000 records from pii-masking-200k (with PII)
  - 43,000 records from Generic Sentiment dataset (without PII)
  - Total: 86,000 balanced records (50% PII, 50% non-PII)
- **Testing Dataset**:
  - pii-masking-43k
  - Dialog Dataset (3,726 records)
  - Movie Review/IMDB (10,000 records)
  - GPT-4 sentences (1,000 records) - Note: This needs to be generated

## Hyperparameters

- Optimizer: adamw_torch
- Learning rate: 5×10⁻⁵
- Weight decay: 0.01
- Batch size: 64
- Epochs: 3
- Train/Val/Test split: 80%/10%/10%
- Cross-validation: 5-fold and stratified 5-fold

## Expected Results

- Accuracy: 99.558%
- Precision: 99.564%
- Recall: 99.558%
- F1-score: 99.559%


## 1. Import Libraries and Set Up Environment


In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

# Suppress the Windows/MacOS redirect warning (it's harmless)
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Check GPU availability and provide information
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("No GPU detected. Training will use CPU (this will be slower).")
    print("Note: For faster training, consider using a GPU-enabled environment.")
    print("      You may want to reduce batch_size if you encounter memory issues.")

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


Using device: cuda
GPU: Tesla T4
CUDA Version: 12.6
GPU Memory: 14.74 GB


In [ ]:
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")


README.md: 0.00B [00:00, ?B/s]

data/train/1english_openpii_30k.jsonl:   0%|          | 0.00/103M [00:00<?, ?B/s]

data/train/dutch_openpii_28k.jsonl:   0%|          | 0.00/102M [00:00<?, ?B/s]

data/train/french_openpii_31k.jsonl:   0%|          | 0.00/114M [00:00<?, ?B/s]

data/train/german_openpii_30k.jsonl:   0%|          | 0.00/108M [00:00<?, ?B/s]

...

data/validation/german_openpii_8k.jsonl:   0%|          | 0.00/29.2M [00:00<?, ?B/s]

data/validation/italian_openpiii_8k.json(…):   0%|          | 0.00/28.3M [00:00<?, ?B/s]

data/validation/spanish_openpii_8k.jsonl:   0%|          | 0.00/27.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/177677 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/47728 [00:00<?, ? examples/s]

## 2. Load Training Datasets

### 2.1 Load pii-masking-200k Dataset


In [ ]:
# Load pii-masking-200k dataset
print("Loading pii-masking-200k dataset...")
pii_200k = load_dataset("ai4privacy/pii-masking-200k")
print(f"Dataset structure: {pii_200k}")

# Get the training split
train_pii = pii_200k['train']
print(f"Total records in pii-masking-200k: {len(train_pii)}")
print(f"Sample record: {train_pii[0]}")

# Extract all records with language 'en'
print("\nFiltering records with language 'en'...")
pii_records = []
for example in train_pii:
    # Check if the language is 'en'
    if example.get('language') == 'en' or example.get('language') == 'de':
        pii_records.append({
            'tokens': example.get('mbert_text_tokens', []),
            'labels': example.get('mbert_bio_labels', []),
            'source_text': example.get('source_text', ''),
            'target_text': example.get('target_text', ''),
            'privacy_mask': example.get('privacy_mask', ''),
            'has_pii': any(label != 'O' for label in example.get('mbert_bio_labels', [])),
            'language' : example.get('language')# Determine if it has PII
        })

print(f"Selected {len(pii_records)} records with language 'en'")
print(f"Records with PII in English subset: {sum(1 for r in pii_records if r['has_pii'])}")
print(f"Records without PII in English subset: {sum(1 for r in pii_records if not r['has_pii'])}")

Loading pii-masking-200k dataset...
Dataset structure: DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 209261
    })
})
Total records in pii-masking-200k: 209261
Sample record: {'source_text': "A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it?", 'target_text': "A student's assessment was found on device bearing IMEI: [PHONEIMEI]. The document falls under the various topics discussed in our [JOBAREA] curriculum. Can you please collect it?", 'privacy_mask': [{'value': '06-184755-866851-3', 'start': 57, 'end': 75, 'label': 'PHONEIMEI'}, {'value': 'Optimization', 'start': 138, 'end': 150, 'label': 'JOBAREA'}], 'span_labels': '[[0, 57, "O"], [57, 75, "PHONEIMEI"], [75, 138, "O"], [138, 150, "JOBAREA"], [

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### 2.2 Load Generic Sentiment Dataset (Non-PII)


In [ ]:
print("\n1. Loading mBERT tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    print("   ✓ mBERT tokenizer loaded successfully!")
    print(f"   Model: bert-base-multilingual-cased")
    print(f"   Tokenizer type: {type(tokenizer).__name__}")
    print(f"   Vocabulary size: {len(tokenizer.vocab)}")
except Exception as e:
    print(f"   ✗ Error loading mBERT tokenizer: {e}")
    print("   Trying to install transformers if needed...")
    sys.exit(1)


1. Loading mBERT tokenizer...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

   ✓ mBERT tokenizer loaded successfully!
   Model: bert-base-multilingual-cased
   Tokenizer type: BertTokenizerFast
   Vocabulary size: 119547


In [ ]:
import sys
# Load Generic Sentiment Dataset (Non-PII)
print("Loading Generic Sentiment Dataset...")

try:
    non_pii_df = pd.read_csv('/content/generic_sentiment_dataset_50k.csv')
    print(f"Loaded {len(non_pii_df)} records from Generic Sentiment Dataset")
    print("Sample record:")
    display(non_pii_df.head())

    # Assuming the non-PII data has a 'text' column
    # Tokenize and format it similar to the PII data
    non_pii_records = []
    for index, row in non_pii_df.iterrows():
        text = str(row.get('text', ''))
        # Tokenize using the mBERT tokenizer
        tokens = tokenizer.tokenize(text)

        # Filter out samples where token length > 512
        if len(tokens) > 512:
            # print(f"Skipping record {index} due to token length > 512: {len(tokens)}")
            continue

        non_pii_records.append({
            'tokens': tokens,
            'labels': ['O'] * len(tokens),  # All labels are 'O' for non-PII
            'source_text': text,
            'has_pii': False
        })

    print(f"Processed {len(non_pii_records)} records for non-PII training data after filtering.")

except FileNotFoundError:
    print("Error: '/content/generic_sentiment_dataset_50k.csv' not found.")
    print("Please upload the dataset file or provide the correct path.")
    non_pii_records = []
except Exception as e:
    print(f"An error occurred while processing the Generic Sentiment Dataset: {e}")
    non_pii_records = []

Loading Generic Sentiment Dataset...
Loaded 50000 records from Generic Sentiment Dataset
Sample record:


,sentiment,text,label
0,positive,good mobile. battery is 5000 mah is very big. ...,2
1,positive,Overall in hand ecpirience is quite good matt ...,2
2,positive,"1. Superb Camera,\n2. No lag\n3. This is my fi...",2
3,positive,Bigger size of application names doesn't allow...,2
4,negative,Just a hype of stock android which is not flaw...,0


Token indices sequence length is longer than the specified maximum sequence length for this model (603 > 512). Running this sequence through the model will result in indexing errors


Processed 49839 records for non-PII training data after filtering.


In [ ]:
import pandas as pd
import csv

# 1. Load dữ liệu tiếng Đức (Code của bạn)
print("Loading German Sentiment Dataset...")
data_frames = []

try:
    # Giả sử file train_v1.4.tsv nằm ở /content/
    df = pd.read_csv(r'/content/train_v1.4.tsv',
                     sep='\t',
                     header=None,
                     names=['url', 'text', 'relevance', 'label', 'aspect'],
                     quoting=csv.QUOTE_NONE,
                     on_bad_lines='skip')
    data_frames.append(df)
except FileNotFoundError:
    print("Error: File not found.")

# 2. Gộp và xử lý
if data_frames:
    full_df = pd.concat(data_frames, ignore_index=True)
    print("-" * 30)
    print(f"Loaded {len(full_df)} records from German Dataset")
    print("Sample raw record:")
    display(full_df.head(1)) # Hiển thị 1 dòng để kiểm tra

    # 3. Xử lý Tokenize và Format (Phần bạn yêu cầu)
    german_non_pii_records = []

    print("\nProcessing records to Non-PII format...")

    for index, row in full_df.iterrows():
        # Lấy text, chuyển về string để tránh lỗi nếu có NaN
        text_content = str(row.get('text', ''))
        tokens = tokenizer.tokenize(text_content)

        if len(tokens) > 512:
            # print(f"Skipping record {index} due to token length > 512: {len(tokens)}")
            continue
        # Bỏ qua nếu dòng không có token nào (dòng trống)
        if not tokens:
            continue

        german_non_pii_records.append({
            'tokens': tokens,
            'labels': ['O'] * len(tokens),  # Tất cả nhãn là 'O' (Outside)
            'source_text': text_content,
            'has_pii': False
        })

    print(f"Processed {len(german_non_pii_records)} records for German non-PII training data")

    # Kiểm tra kết quả sau khi xử lý
    if len(german_non_pii_records) > 0:
        print("\nSample processed record:")
        print(german_non_pii_records[0])

else:
    print("Không có dữ liệu để xử lý.")

Loading German Sentiment Dataset...
------------------------------
Loaded 20941 records from German Dataset
Sample raw record:


,url,text,relevance,label,aspect
0,http://twitter.com/reneesa\_devin/statuses/628...,"@DB_Bahn ja, weil in Wuppertal Bauarbeiten sin...",True,neutral,Allgemein#Haupt:neutral



Processing records to Non-PII format...
Processed 20037 records for German non-PII training data

Sample processed record:
{'tokens': ['@', 'DB', '_', 'Bahn', 'ja', ',', 'weil', 'in', 'Wuppertal', 'Bau', '##arbeiten', 'sind', ',', 'so', '##weit', 'bin', 'ich', 'auch', ',', 'aber', 'wies', '##o', 'nur', 'am', 'Wochen', '##ende', 'und', 'grade', 'jetzt', '?'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'source_text': '@DB_Bahn ja, weil in Wuppertal Bauarbeiten sind, soweit bin ich auch, aber wieso nur am Wochenende und grade jetzt?', 'has_pii': False}


### 2.3 Combine Training Datasets


In [ ]:
vn_pii = pd.read_csv('/content/dong10000_32000_data.csv')

In [ ]:
vn_pii.head()

,source_text,target_text,tokens,labels
0,Cơ sở dữ liệu tiêm chủng nhân viên sẽ được duy...,Cơ sở dữ liệu tiêm chủng nhân viên sẽ được duy...,"['Cơ', 'sở', 'dữ', 'liệu', 'ti', '##êm', 'chủn...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"Chào phong.cao62, buổi tư vấn chấn thương của ...","Chào [USERNAME], buổi tư vấn chấn thương của b...","['Ch', '##ào', 'phong', '.', 'cao', '##6', '##...","['O', 'O', 'B-USERNAME', 'I-USERNAME', 'I-USER..."
2,"Chào Mỹ, công ty chúng tôi đang lên kế hoạch g...","Chào [FIRSTNAME], công ty chúng tôi đang lên k...","['Ch', '##ào', 'Mỹ', ',', 'công', 'ty', 'chúng...","['O', 'O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', ..."
3,"Chào Yên, đang tiến hành một khảo sát trực tuy...","Chào [FIRSTNAME], đang tiến hành một khảo sát ...","['Ch', '##ào', 'Yên', ',', 'đang', 'tiến', 'hà...","['O', 'O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', ..."
4,"Chào Việt, báo cáo xét nghiệm của bạn từ khoa ...","Chào [FIRSTNAME], báo cáo xét nghiệm của bạn t...","['Ch', '##ào', 'Việt', ',', 'báo', 'cáo', 'xét...","['O', 'O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', ..."


In [ ]:
import ast

# Assuming pii_records is already defined as a list (e.g., from previous steps or initialized as empty)
# If not, initialize it:
pii_vi = []

print("Processing Vietnamese PII records from vn_pii DataFrame...")

processed_vn_pii_records = []
for index, row in vn_pii.iterrows():
    try:
        tokens_list = ast.literal_eval(row['tokens'])
        labels_list = ast.literal_eval(row['labels'])
        processed_vn_pii_records.append({
            'tokens': tokens_list,
            'labels': labels_list,
            'source_text': row['source_text'],
            'has_pii': True
        })
    except (ValueError, SyntaxError) as e:
        print(f"Skipping row {index} due to error parsing tokens or labels: {e}")

# Append these processed records to the existing pii_records list
# If pii_records was intended to *only* contain these VN records, you might reassign instead of extend.
# For now, following the instruction to 'append these processed Vietnamese PII records to the `pii_records` list.'
# Ensure pii_records is initialized before this step if it's not coming from previous cell outputs.

# In the provided notebook context, pii_records was emptied or not yet populated with English/German records.
# To combine all PII records, we will extend it. Assuming pii_records currently holds English/German PII if loaded.
# To avoid duplicating English/German records if this cell is run multiple times,
# and given the instruction to append *these* to *pii_records*, we will assume pii_records is in a state to be extended.

# If pii_records was previously cleared and only this subtask is filling it, this is fine.
# If it already had content and this is meant to *add* to it, ensure it's not being overwritten by mistake in other cells.
# For the purpose of this subtask (appending to existing pii_records), let's ensure it's a list.
if 'pii_records' not in locals() or not isinstance(pii_vi, list):
    pii_vi = [] # Initialize if not already a list

pii_vi.extend(processed_vn_pii_records)

print(f"Appended {len(processed_vn_pii_records)} Vietnamese PII records to pii_records. Total PII records now: {len(pii_vi)}")

# Print a sample to verify
if pii_vi:
    print("\nSample processed Vietnamese PII record:")
    print(pii_vi[0])

Processing Vietnamese PII records from vn_pii DataFrame...
Appended 34203 Vietnamese PII records to pii_records. Total PII records now: 34203

Sample processed Vietnamese PII record:
{'tokens': ['Cơ', 'sở', 'dữ', 'liệu', 'ti', '##êm', 'chủng', 'nhân', 'viên', 'sẽ', 'được', 'duy', 'trì', 'bởi', 'N', '##ông', 'nghiệp', '.', 'T', '##ất', 'cả', 'các', 'mục', 'sẽ', 'được', 'xác', 'minh', 'bằng', 'giấy', 't', '##ờ', 'tù', '##y', 'thân', 'có', 'ảnh', 'và', '[', 'SS', '##N', ']', '.', 'Ph', '##ối', 'hợp', 'các', 'n', '##ỗ', 'lực', 'qua', '06', '-', '367', '##7', '##6', '##1', '-', '556', '##69', '##9', '-', '8', '.'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-JOBAREA', 'I-JOBAREA', 'I-JOBAREA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHON

In [ ]:
vi_non_pii = pd.read_csv('/content/data.csv')

In [ ]:
vi_non_pii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31460 entries, 0 to 31459
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  31436 non-null  object
 1   label    31460 non-null  object
 2   start    31460 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 737.5+ KB


In [ ]:
print("\nProcessing Vietnamese non-PII records from vi_non_pii DataFrame...")

vietnamese_non_pii_records = []

# Take up to 10,000 samples as requested by the user
num_samples_to_take = 31430

for index, row in vi_non_pii.head(num_samples_to_take).iterrows():
    text = str(row.get('content', '')) # Assuming 'content' column for text

    # Tokenize using the mBERT tokenizer
    tokens = tokenizer.tokenize(text)

    # Filter out samples where token length > 512
    if len(tokens) > 512:
        continue

    # Skip if no tokens are generated (e.g., empty string)
    if not tokens:
        continue

    vietnamese_non_pii_records.append({
        'tokens': tokens,
        'labels': ['O'] * len(tokens),  # All labels are 'O' for non-PII
        'source_text': text,
        'has_pii': False
    })

print(f"Processed {len(vietnamese_non_pii_records)} records for Vietnamese non-PII training data after filtering and sampling.")

# Print a sample to verify
if vietnamese_non_pii_records:
    print("\nSample processed Vietnamese non-PII record:")
    print(vietnamese_non_pii_records[0])


Processing Vietnamese non-PII records from vi_non_pii DataFrame...
Processed 31430 records for Vietnamese non-PII training data after filtering and sampling.

Sample processed Vietnamese non-PII record:
{'tokens': ['Áo', 'bao', 'đẹp', 'ạ', '!'], 'labels': ['O', 'O', 'O', 'O', 'O'], 'source_text': 'Áo bao đẹp ạ!', 'has_pii': False}


In [ ]:
# Combine PII and non-PII records
all_training_records = pii_vi + vietnamese_non_pii_records + pii_records + german_non_pii_records + non_pii_records # Balance the dataset

print(f"Total training records: {len(all_training_records)}")
print(f"Records with PII: {sum(1 for r in all_training_records if r['has_pii'])}")
print(f"Records without PII: {sum(1 for r in all_training_records if not r['has_pii'])}")

# Shuffle the dataset
import random
random.shuffle(all_training_records)

# Create a dataset from the records
training_dataset = Dataset.from_list(all_training_records)
print(f"\nTraining dataset created: {training_dataset}")
print(f"Features: {training_dataset.features}")


Total training records: 231827
Records with PII: 130521
Records without PII: 101306

Training dataset created: Dataset({
    features: ['tokens', 'labels', 'source_text', 'has_pii'],
    num_rows: 231827
})
Features: {'tokens': List(Value('string')), 'labels': List(Value('string')), 'source_text': Value('string'), 'has_pii': Value('bool')}


In [ ]:
train_testvalid = all_training_records.train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

AttributeError: 'list' object has no attribute 'train_test_split'

In [ ]:
# Simple EDA on the training_dataset
print("Training dataset structure:")
print(training_dataset)
print("\nTraining dataset features:")
print(training_dataset.features)
print(f"\nNumber of records in training dataset: {len(training_dataset)}")
print("\nSample record from training dataset:")
print(training_dataset[0])

# You can also convert to pandas DataFrame for more extensive EDA if needed
# try:
#     training_df = training_dataset.to_pandas()
#     print("\nConverted training dataset to pandas DataFrame:")
#     display(training_df.head())
# except Exception as e:
#     print(f"\nCould not convert training dataset to pandas DataFrame: {e}")

Training dataset structure:
Dataset({
    features: ['tokens', 'labels', 'source_text', 'has_pii'],
    num_rows: 231827
})

Training dataset features:
{'tokens': List(Value('string')), 'labels': List(Value('string')), 'source_text': Value('string'), 'has_pii': Value('bool')}

Number of records in training dataset: 231827

Sample record from training dataset:
{'tokens': ['As', 'a', 'real', 'estate', 'in', '##vestor', ',', 'I', 'need', 'contract', '##ors', 'that', 'I', 'can', 'truly', 'count', 'on', 'to', 'get', 'things', 'done', 'for', 'me', '.', 'This', 'means', 'that', 'my', 'calls', 'get', 'returned', 'in', 'a', 'time', '##ly', 'fashion', ',', 'work', 'is', 'done', 'properly', ',', 'quickly', 'and', 'compete', '##ntly', 'but', 'most', 'important', '##ly', ',', 'that', 'I', 'get', 'quoted', 'a', 'fair', 'price', 'for', 'a', 'good', 'job', '.', 'Don', "'", 't', 'give', 'me', 'drama', 'and', 'def', '##inite', '##ly', ',', 'don', "'", 't', 'give', 'me', 'any', 'bu', '##lls', '##hit', 'e

## 3. Load BERT Model and Tokenizer


In [ ]:
# Load BERT model and tokenizer
# Using bert-base-uncased as it's commonly used and has ~110M parameters
# For 340M parameters, we might need bert-large-uncased or a custom model
model_name = "distilbert/distilbert-base-multilingual-cased"  # You can change this to "bert-large-uncased" for more parameters

print(f"Loading model: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Get all unique labels from the training data
all_labels = set()
for record in all_training_records:
    all_labels.update(record['labels'])

label_list = sorted(list(all_labels))
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {idx: label for idx, label in enumerate(label_list)}

print(f"Number of labels: {len(label_list)}")
print(f"Labels: {label_list[:20]}...")  # Show first 20 labels

# Load model for token classification
num_labels = len(label_list)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

print(f"Model loaded. Number of parameters: {sum(p.numel() for p in model.parameters()):,}")


Loading model: distilbert/distilbert-base-multilingual-cased


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Number of labels: 115
Labels: ['B-ACCOUNTNAME', 'B-ACCOUNTNUMBER', 'B-AGE', 'B-AMOUNT', 'B-BIC', 'B-BITCOINADDRESS', 'B-BUILDINGNUMBER', 'B-CCCD', 'B-CITY', 'B-COMPANYNAME', 'B-COUNTY', 'B-CREDITCARDCVV', 'B-CREDITCARDISSUER', 'B-CREDITCARDNUMBER', 'B-CURRENCY', 'B-CURRENCYCODE', 'B-CURRENCYNAME', 'B-CURRENCYSYMBOL', 'B-DATE', 'B-DOB']...


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded. Number of parameters: 134,822,515


## 4. Preprocess Data for BERT


In [ ]:
def process_data_for_mbert(examples):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []

    # Duyệt qua từng mẫu trong batch
    for tokens, original_labels in zip(examples['tokens'], examples['labels']):
        # 1. Chuyển Tokens thành IDs
        # Vì tokens đã là subword của mBERT, ta dùng convert_tokens_to_ids
        ids = tokenizer.convert_tokens_to_ids(tokens)

        # 2. Xử lý nhãn
        # Chuyển nhãn chuỗi (B-PER...) sang ID số
        # Lưu ý: original_labels đang là chuỗi, cần map sang số
        label_ids = [label_to_id[l] for l in original_labels]

        # 3. Thêm Special Tokens ([CLS] và [SEP]) và Truncate
        # Giới hạn độ dài là 512. Trừ 2 cho [CLS] và [SEP]
        max_len = 512 - 2
        if len(ids) > max_len:
            ids = ids[:max_len]
            label_ids = label_ids[:max_len]

        # Thêm [CLS] (101) vào đầu và [SEP] (102) vào cuối
        final_input_ids = [tokenizer.cls_token_id] + ids + [tokenizer.sep_token_id]

        # Thêm nhãn cho [CLS] và [SEP] là -100 (để PyTorch bỏ qua khi tính loss)
        final_label_ids = [-100] + label_ids + [-100]

        # Tạo attention mask (1 cho token thật)
        final_attention_mask = [1] * len(final_input_ids)

        input_ids_list.append(final_input_ids)
        labels_list.append(final_label_ids)
        attention_mask_list.append(final_attention_mask)

    # Trả về định dạng mà Trainer mong muốn
    return {
        "input_ids": input_ids_list,
        "labels": labels_list,
        "attention_mask": attention_mask_list
    }

# Áp dụng hàm mới
print("Processing data direct mapping...")
tokenized_dataset = training_dataset.map(
    process_data_for_mbert,
    batched=True,
    remove_columns=training_dataset.column_names # Xóa cột cũ để đỡ tốn RAM
)

print(f"Sample processed: {tokenized_dataset[0]}")

Processing data direct mapping...


Map:   0%|          | 0/231827 [00:00<?, ? examples/s]

Sample processed: {'labels': [-100, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114,

## 5. Split Data into Train/Validation/Test Sets


In [ ]:
# Split dataset: 80% train, 10% validation, 10% test
train_testvalid = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

train_dataset = train_testvalid['train']
val_dataset = test_valid['train']
test_dataset = test_valid['test']

print(f"Training set: {len(train_dataset)} samples")
print(f"Validation set: {len(val_dataset)} samples")
print(f"Test set: {len(test_dataset)} samples")


Training set: 185461 samples
Validation set: 23183 samples
Test set: 23183 samples


## 6. Define Evaluation Metrics


In [ ]:
def compute_metrics(eval_pred):
    """Compute metrics for evaluation"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten lists
    true_predictions_flat = [item for sublist in true_predictions for item in sublist]
    true_labels_flat = [item for sublist in true_labels for item in sublist]

    # Calculate metrics
    accuracy = accuracy_score(true_labels_flat, true_predictions_flat)
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels_flat, true_predictions_flat, average='weighted', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


## 7. Training Configuration


In [ ]:
# Training arguments as specified in the paper
# Adjust batch size for CPU if needed (smaller batch size for CPU to avoid memory issues)
batch_size = 32 if torch.cuda.is_available() else 16  # Smaller batch for CPU
eval_batch_size = 32 if torch.cuda.is_available() else 16

training_args = TrainingArguments(
    output_dir="./pii_detection_model",
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    learning_rate=5e-5,
    weight_decay=0.01,
    optim="adamw_torch",
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=2,
    seed=42,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
    dataloader_num_workers=0 if not torch.cuda.is_available() else 4,  # Reduce workers for CPU
    report_to="none",  # Disable wandb/tensorboard if not needed
)

# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

print("Training configuration:")
print(f"  Device: {device}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size} (adjusted for {device})")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Weight decay: {training_args.weight_decay}")
print(f"  Optimizer: {training_args.optim}")
print(f"  Mixed precision (fp16): {training_args.fp16}")
if not torch.cuda.is_available():
    print("\n⚠️  Note: Training on CPU will be significantly slower.")
    print("   Estimated time: Several hours to days depending on dataset size.")
    print("   Consider using Google Colab (free GPU) or a cloud GPU instance for faster training.")

Training configuration:
  Device: cuda
  Epochs: 3
  Batch size: 32 (adjusted for cuda)
  Learning rate: 5e-05
  Weight decay: 0.01
  Optimizer: OptimizerNames.ADAMW_TORCH
  Mixed precision (fp16): True


In [ ]:
model_path = "/content/drive/MyDrive/pii_multi_v2"

print(f"Loading tokenizer from: {model_path}")
tokenizer = AutoTokenizer.from_pretrained(model_path)

print(f"Loading model from: {model_path}")
model = AutoModelForTokenClassification.from_pretrained(model_path)

print("Model and tokenizer loaded successfully.")

Loading tokenizer from: /content/drive/MyDrive/pii_multi_v2


The tokenizer you are loading from '/content/drive/MyDrive/pii_multi_v2' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading model from: /content/drive/MyDrive/pii_multi_v2
Model and tokenizer loaded successfully.


In [ ]:
# Prepare data collator for token classification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

print("Data collator created successfully!")
print(f"Number of labels in model: {model.config.num_labels}")
print(f"Number of labels in our label list: {len(label_list)}")

# Update model config if needed
if model.config.num_labels != len(label_list):
    print(f"Warning: Model has {model.config.num_labels} labels but dataset has {len(label_list)} labels")
    print("Creating new model with correct number of labels...")
    from transformers import AutoConfig
    config = AutoConfig.from_pretrained(model_path)
    config.num_labels = len(label_list)
    config.id2label = id_to_label
    config.label2id = {v: k for k, v in id_to_label.items()}

    # Reload model with correct config
    model = AutoModelForTokenClassification.from_pretrained(
        model_path,
        config=config,
        ignore_mismatched_sizes=True
    )
    print("Model reloaded with correct number of labels!")
else:
    # Update config to match our label mappings
    model.config.id2label = id_to_label
    model.config.label2id = {v: k for k, v in id_to_label.items()}
    print("Model config updated with label mappings!")

print(f"\nModel ready for evaluation!")


Data collator created successfully!
Number of labels in model: 115
Number of labels in our label list: 115
Model config updated with label mappings!

Model ready for evaluation!


In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Evaluate on test set
print("\nEvaluating on test set...")
test_results = trainer.evaluate(test_dataset)
print(f"\nTest Results:")
print(f"  Accuracy: {test_results['eval_accuracy']:.6f} ({test_results['eval_accuracy']*100:.3f}%)")
print(f"  Precision: {test_results['eval_precision']:.6f} ({test_results['eval_precision']*100:.3f}%)")
print(f"  Recall: {test_results['eval_recall']:.6f} ({test_results['eval_recall']*100:.3f}%)")
print(f"  F1-Score: {test_results['eval_f1']:.6f} ({test_results['eval_f1']*100:.3f}%)")

# Save the final model
trainer.save_model("./pii_detection_model_final")
print("\nModel saved to ./pii_detection_model_final")



Evaluating on test set...



Test Results:
  Accuracy: 0.022010 (2.201%)
  Precision: 0.653891 (65.389%)
  Recall: 0.022010 (2.201%)
  F1-Score: 0.040856 (4.086%)

Model saved to ./pii_detection_model_final


In [ ]:
import pandas as pd
import numpy as np

print("Getting predictions from the test dataset...")

# Get predictions from the trainer on the test_dataset
predictions_output = trainer.predict(test_dataset)

# Extract predictions (logits) and true labels
predictions = predictions_output.predictions
true_labels = predictions_output.label_ids

# Convert prediction logits to predicted label IDs
predicted_label_ids = np.argmax(predictions, axis=2)

# Initialize lists to store flattened true and predicted labels
all_true_labels = []
all_predicted_labels = []

# Define a mapping function to group IP-related labels
def group_ip_labels(label):
    if 'IP' in label and ('IPV4' in label or 'IPV6' in label or label.endswith('IP')):
        if label.startswith('B-'):
            return 'B-IP'
        elif label.startswith('I-'):
            return 'I-IP'
        else:
            return 'IP' # Should not happen with B/I schema, but as a fallback
    return label

print("Processing predictions and true labels...")

# Iterate through each sample to align and collect true and predicted labels
for sample_true_labels, sample_predicted_ids in zip(true_labels, predicted_label_ids):
    for true_id, predicted_id in zip(sample_true_labels, sample_predicted_ids):
        # We only care about tokens that were not ignored (-100 label)
        if true_id != -100:
            original_true_label = id_to_label[true_id]
            original_predicted_label = id_to_label[predicted_id]

            all_true_labels.append(group_ip_labels(original_true_label))
            all_predicted_labels.append(group_ip_labels(original_predicted_label))

print(f"Total labels processed: {len(all_true_labels)}")

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'True Label': all_true_labels,
    'Predicted Label': all_predicted_labels
})

# Calculate misclassification counts
misclassification_counts = comparison_df.groupby(['True Label', 'Predicted Label']).size().reset_index(name='Count')

# Sort by count in descending order
misclassification_counts = misclassification_counts.sort_values(by='Count', ascending=False)

print("\nTop 20 True vs. Predicted Label Counts (including correct predictions):")
print(misclassification_counts.head(20))

print("\nMisclassifications (where True Label != Predicted Label) - Top 20:")
misclassification_df = misclassification_counts[misclassification_counts['True Label'] != misclassification_counts['Predicted Label']]
print(misclassification_df.head(20))

# Optionally, calculate per-label accuracy/precision/recall from this table
# For example, to find accuracy for a specific label, say 'B-PERSON'
# correct_predictions_B_PERSON = misclassification_counts[(misclassification_counts['True Label'] == 'B-PERSON') & (misclassification_counts['Predicted Label'] == 'B-PERSON')]['Count'].sum()
# total_true_B_PERSON = misclassification_counts[misclassification_counts['True Label'] == 'B-PERSON']['Count'].sum()
# accuracy_B_PERSON = correct_predictions_B_PERSON / total_true_B_PERSON if total_true_B_PERSON > 0 else 0
# print(f"Accuracy for B-PERSON: {accuracy_B_PERSON:.4f}")

Getting predictions from the test dataset...


Processing predictions and true labels...
Total labels processed: 1086545

Top 20 True vs. Predicted Label Counts (including correct predictions):
                True Label        Predicted Label   Count
474                      O                      O  816671
297                   I-IP                   I-IP   36887
379            I-USERAGENT            I-USERAGENT   25805
208       I-BITCOINADDRESS       I-BITCOINADDRESS   19774
275      I-ETHEREUMADDRESS      I-ETHEREUMADDRESS   14948
272                I-EMAIL                I-EMAIL   10388
377                  I-URL                  I-URL    9079
293                 I-IBAN                 I-IBAN    8782
343             I-PASSWORD             I-PASSWORD    7175
245     I-CREDITCARDNUMBER     I-CREDITCARDNUMBER    6110
345            I-PHONEIMEI            I-PHONEIMEI    5764
340  I-NEARBYGPSCOORDINATE  I-NEARBYGPSCOORDINATE    5758
261                 I-DATE                 I-DATE    5560
322      I-LITECOINADDRESS      I-LITECOI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import ast
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix, classification_report

# ==========================================
# 1. CẤU HÌNH
# ==========================================
MODEL_PATH = "/content/drive/MyDrive/pii_multi_v2" # Đường dẫn model đã train của bạn
FALLBACK_MODEL = "google-bert/bert-base-multilingual-cased"
DATA_FILE = r'/content/vietnam_pii.csv'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. LOAD MODEL & DATA
# ==========================================
print("Loading resources...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)
    print("✅ Loaded Fine-tuned Model.")
except:
    print("⚠️ Loading Base Model (Demo mode)...")
    tokenizer = AutoTokenizer.from_pretrained(FALLBACK_MODEL)
    model = AutoModelForTokenClassification.from_pretrained(FALLBACK_MODEL, num_labels=20) # Demo num_labels

model.to(device)
label2id = model.config.label2id
id2label = model.config.id2label
label_list = list(label2id.keys())

# Xử lý dữ liệu
df = pd.read_csv(DATA_FILE)
df['tokens'] = df['tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['labels'] = df['labels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df = df[df['tokens'].map(len) > 0]
dataset = Dataset.from_pandas(df)

# Tokenize function
def tokenize_and_align(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        is_split_into_words=True, # Đã là sub-words nên True hay False cần cẩn trọng.
        # NOTE: Vì input của bạn LÀ SUB-WORD rồi, ta dùng convert_tokens_to_ids thủ công sẽ chuẩn hơn
        # như code trước. Nhưng để dùng Trainer.predict tiện lợi, ta cần input_ids.
        # Đoạn dưới đây dùng logic map thủ công ID như code đánh giá trước:
        truncation=True,
        max_length=512
    )

    # Ghi đè input_ids bằng cách convert thủ công để khớp 1-1
    all_input_ids = []
    all_labels = []
    skipped_count = 0  # Đếm số samples bị bỏ qua

    for i, tokens in enumerate(examples['tokens']):
        try:
            # Convert tokens -> ids
            ids = tokenizer.convert_tokens_to_ids(tokens)

            # Xử lý labels tương ứng
            lbls = [label2id.get(l, 0) for l in examples['labels'][i]]

            # QUAN TRỌNG: Đảm bảo ids và lbls có cùng độ dài
            min_len = min(len(ids), len(lbls))
            ids = ids[:min_len]
            lbls = lbls[:min_len]

            # Cắt/Pad và thêm CLS/SEP
            if len(ids) > 510:
                ids = ids[:510]
                lbls = lbls[:510]

            ids = [tokenizer.cls_token_id] + ids + [tokenizer.sep_token_id]
            lbls = [-100] + lbls + [-100]

            # Kiểm tra lại độ dài cuối cùng
            if len(ids) != len(lbls):
                skipped_count += 1
                continue  # Bỏ qua sample này

            all_input_ids.append(ids)
            all_labels.append(lbls)

        except Exception as e:
            # Bỏ qua sample nếu có lỗi
            skipped_count += 1
            continue

    if skipped_count > 0:
        print(f"⚠️ Đã bỏ qua {skipped_count} samples bị lỗi")

    return {"input_ids": all_input_ids, "labels": all_labels}

tokenized_dataset = dataset.map(tokenize_and_align, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# ==========================================
# 3. DỰ ĐOÁN (PREDICT)
# ==========================================
trainer = Trainer(model=model, data_collator=data_collator)
print("Running prediction...")
predictions, labels, _ = trainer.predict(tokenized_dataset)
predictions = np.argmax(predictions, axis=2)

# ==========================================
# 4. TẠO BÁO CÁO & MA TRẬN
# ==========================================
# Flatten lists và loại bỏ -100
true_labels = []
pred_labels = []

for i in range(len(labels)):
    for j in range(len(labels[i])):
        if labels[i][j] != -100:
            true_labels.append(id2label[labels[i][j]])
            pred_labels.append(id2label[predictions[i][j]])

# A. Báo cáo chi tiết (Accuracy từng label)
print("\n" + "="*50)
print("BÁO CÁO CHI TIẾT TỪNG NHÃN (CLASSIFICATION REPORT)")
print("="*50)
print(classification_report(true_labels, pred_labels, zero_division=0))

# B. Ma trận nhầm lẫn (Confusion Matrix)
labels_unique = sorted(list(set(true_labels + pred_labels)))
cm = confusion_matrix(true_labels, pred_labels, labels=labels_unique)

# Tạo DataFrame cho dễ nhìn
cm_df = pd.DataFrame(cm, index=labels_unique, columns=labels_unique)

print("\n" + "="*50)
print("MA TRẬN NHẦM LẪN (CONFUSION MATRIX)")
print("Hàng: Thực tế (True) | Cột: Dự đoán (Predicted)")
print("="*50)
# Hiển thị bảng (nếu nhiều nhãn quá sẽ bị cắt, bạn có thể export ra excel)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(cm_df)

# C. Xuất ra Excel để xem kỹ hơn (nếu cần)
# cm_df.to_csv("confusion_matrix.csv")
# print("\nĐã lưu ma trận vào file 'confusion_matrix.csv'")

Loading resources...


The tokenizer you are loading from '/content/drive/MyDrive/pii_multi_v2' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ Loaded Fine-tuned Model.


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Running prediction...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"



BÁO CÁO CHI TIẾT TỪNG NHÃN (CLASSIFICATION REPORT)
                    precision    recall  f1-score   support

   B-ACCOUNTNUMBER       0.33      0.67      0.44         6
  B-BUILDINGNUMBER       1.00      0.86      0.92         7
            B-CCCD       0.00      0.00      0.00         0
            B-CITY       0.45      0.42      0.43        12
     B-COMPANYNAME       0.77      0.71      0.74        14
             B-DOB       1.00      1.00      1.00         6
           B-EMAIL       1.00      1.00      1.00        13
       B-FIRSTNAME       0.62      0.82      0.71        34
         B-JOBAREA       0.00      0.00      0.00         0
        B-LASTNAME       0.86      0.91      0.89        34
    B-MASKEDNUMBER       0.00      0.00      0.00         0
      B-MIDDLENAME       0.90      0.56      0.69        34
     B-PHONENUMBER       0.96      0.88      0.92        25
             B-SSN       0.00      0.00      0.00        11
           B-STATE       0.30      1.00      0.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

def merge_name_labels(labels):
    """
    Gộp FIRSTNAME, MIDDLENAME, LASTNAME thành NAME

    Args:
        labels: List các labels (có thể là string hoặc list)

    Returns:
        List labels đã được gộp
    """
    if isinstance(labels, str):
        # Nếu là string, chuyển thành list trước
        labels = [labels]

    merged = []
    for label in labels:
        if label.startswith('B-'):
            entity_type = label[2:]
            if entity_type in ['FIRSTNAME', 'MIDDLENAME', 'LASTNAME']:
                merged.append('B-NAME')
            else:
                merged.append(label)
        elif label.startswith('I-'):
            entity_type = label[2:]
            if entity_type in ['FIRSTNAME', 'MIDDLENAME', 'LASTNAME']:
                merged.append('I-NAME')
            else:
                merged.append(label)
        else:
            merged.append(label)

    return merged if len(merged) > 1 or not isinstance(labels, str) else merged[0]

def evaluate_with_merged_names(true_labels, pred_labels):
    """
    Đánh giá lại với labels đã gộp NAME

    Args:
        true_labels: List labels thực tế
        pred_labels: List labels dự đoán

    Returns:
        Dictionary chứa metrics và confusion matrix
    """
    # Gộp labels
    true_labels_merged = merge_name_labels(true_labels)
    pred_labels_merged = merge_name_labels(pred_labels)

    # Tính metrics
    accuracy = accuracy_score(true_labels_merged, pred_labels_merged)
    precision, recall, f1, support = precision_recall_fscore_support(
        true_labels_merged, pred_labels_merged,
        average='weighted', zero_division=0
    )

    # Confusion matrix
    labels_unique = sorted(list(set(true_labels_merged + pred_labels_merged)))
    cm = confusion_matrix(true_labels_merged, pred_labels_merged, labels=labels_unique)

    # Classification report
    report = classification_report(
        true_labels_merged, pred_labels_merged,
        labels=labels_unique,
        zero_division=0,
        output_dict=True
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm,
        'labels': labels_unique,
        'classification_report': report,
        'true_labels_merged': true_labels_merged,
        'pred_labels_merged': pred_labels_merged
    }

def print_merged_evaluation(results):
    """
    In kết quả đánh giá với labels đã gộp
    """
    print("=" * 80)
    print("KẾT QUẢ ĐÁNH GIÁ VỚI NAME (GỘP FIRSTNAME, MIDDLENAME, LASTNAME)")
    print("=" * 80)
    print(f"\nAccuracy : {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
    print(f"Precision: {results['precision']:.4f} ({results['precision']*100:.2f}%)")
    print(f"Recall   : {results['recall']:.4f} ({results['recall']*100:.2f}%)")
    print(f"F1-Score : {results['f1']:.4f} ({results['f1']*100:.2f}%)")

    print(f"\n{'='*80}")
    print("BÁO CÁO CHI TIẾT TỪNG NHÃN (SAU KHI GỘP NAME)")
    print(f"{'='*80}")

    # In classification report dạng text
    report_text = classification_report(
        results['true_labels_merged'],
        results['pred_labels_merged'],
        labels=results['labels'],
        zero_division=0
    )
    print(report_text)

    # Confusion matrix
    print(f"\n{'='*80}")
    print("MA TRẬN NHẦM LẪN (SAU KHI GỘP NAME)")
    print(f"{'='*80}")
    cm_df = pd.DataFrame(
        results['confusion_matrix'],
        index=results['labels'],
        columns=results['labels']
    )
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    print("\nHàng = Label thực tế, Cột = Label dự đoán")
    print(cm_df)

# Ví dụ sử dụng
print("Hàm đánh giá với NAME đã được định nghĩa!")
print("\nCách sử dụng:")
print("1. Gọi evaluate_with_merged_names(true_labels, pred_labels)")
print("2. Gọi print_merged_evaluation(results) để xem báo cáo")


Hàm đánh giá với NAME đã được định nghĩa!

Cách sử dụng:
1. Gọi evaluate_with_merged_names(true_labels, pred_labels)
2. Gọi print_merged_evaluation(results) để xem báo cáo


In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from sklearn.metrics import confusion_matrix, classification_report

def analyze_label_errors(true_labels, pred_labels, top_n=10):
    """
    Phân tích các label bị sai và xem chúng bị gán vào label nào

    Args:
        true_labels: List các label thực tế
        pred_labels: List các label dự đoán
        top_n: Số lượng lỗi nhầm lẫn phổ biến nhất cần hiển thị

    Returns:
        Dictionary chứa các thống kê về lỗi
    """
    errors = []
    error_stats = defaultdict(lambda: defaultdict(int))

    # Thu thập tất cả các lỗi
    for true_label, pred_label in zip(true_labels, pred_labels):
        if true_label != pred_label:
            errors.append((true_label, pred_label))
            error_stats[true_label][pred_label] += 1

    # Tính toán thống kê
    total_errors = len(errors)
    total_samples = len(true_labels)
    accuracy = (total_samples - total_errors) / total_samples if total_samples > 0 else 0

    # Tìm các lỗi nhầm lẫn phổ biến nhất
    error_counter = Counter(errors)
    top_errors = error_counter.most_common(top_n)

    # Tạo báo cáo chi tiết cho từng label
    label_error_reports = {}
    for true_label in sorted(set(true_labels)):
        if true_label in error_stats:
            total_errors_for_label = sum(error_stats[true_label].values())
            label_error_reports[true_label] = {
                'total_errors': total_errors_for_label,
                'most_common_errors': sorted(
                    error_stats[true_label].items(),
                    key=lambda x: x[1],
                    reverse=True
                )[:top_n]
            }

    return {
        'total_samples': total_samples,
        'total_errors': total_errors,
        'accuracy': accuracy,
        'top_errors': top_errors,
        'error_stats': dict(error_stats),
        'label_error_reports': label_error_reports
    }

def print_error_analysis(error_analysis):
    """
    In ra báo cáo phân tích lỗi một cách dễ đọc
    """
    print("=" * 80)
    print("PHÂN TÍCH CÁC LABEL BỊ SAI")
    print("=" * 80)
    print(f"\nTổng số mẫu: {error_analysis['total_samples']:,}")
    print(f"Tổng số lỗi: {error_analysis['total_errors']:,}")
    print(f"Độ chính xác: {error_analysis['accuracy']:.4f} ({error_analysis['accuracy']*100:.2f}%)")
    print(f"Tỷ lệ lỗi: {(1 - error_analysis['accuracy']):.4f} ({(1 - error_analysis['accuracy'])*100:.2f}%)")

    # Top errors
    print(f"\n{'='*80}")
    print(f"TOP {len(error_analysis['top_errors'])} LỖI NHẦM LẪN PHỔ BIẾN NHẤT")
    print(f"{'='*80}")
    print(f"{'Thực tế':<30} {'Dự đoán':<30} {'Số lần':<10}")
    print("-" * 80)
    for (true_label, pred_label), count in error_analysis['top_errors']:
        print(f"{true_label:<30} {pred_label:<30} {count:<10}")

    # Chi tiết từng label
    print(f"\n{'='*80}")
    print("CHI TIẾT LỖI THEO TỪNG LABEL (chỉ hiển thị labels có lỗi)")
    print(f"{'='*80}")

    for label in sorted(error_analysis['label_error_reports'].keys()):
        report = error_analysis['label_error_reports'][label]
        print(f"\n[{label}]")
        print(f"  Tổng số lỗi: {report['total_errors']}")
        print(f"  Các label bị nhầm lẫn phổ biến nhất:")
        for pred_label, count in report['most_common_errors']:
            percentage = (count / report['total_errors']) * 100
            print(f"    → {pred_label:<30} {count:>5} lần ({percentage:>5.1f}%)")

def create_error_confusion_matrix(true_labels, pred_labels, labels_list=None):
    """
    Tạo confusion matrix chỉ cho các lỗi (bỏ qua các dự đoán đúng)
    """
    if labels_list is None:
        labels_list = sorted(set(true_labels + pred_labels))

    # Tạo confusion matrix đầy đủ
    cm = confusion_matrix(true_labels, pred_labels, labels=labels_list)

    # Tạo confusion matrix chỉ cho lỗi (set diagonal = 0)
    error_cm = cm.copy()
    np.fill_diagonal(error_cm, 0)

    return error_cm, labels_list

# Ví dụ sử dụng với dữ liệu từ cell trước
print("Hàm phân tích lỗi đã được định nghĩa!")
print("\nCách sử dụng:")
print("1. Gọi analyze_label_errors(true_labels, pred_labels)")
print("2. Gọi print_error_analysis(result) để xem báo cáo")
print("3. Gọi create_error_confusion_matrix() để xem ma trận lỗi")


Hàm phân tích lỗi đã được định nghĩa!

Cách sử dụng:
1. Gọi analyze_label_errors(true_labels, pred_labels)
2. Gọi print_error_analysis(result) để xem báo cáo
3. Gọi create_error_confusion_matrix() để xem ma trận lỗi


In [ ]:
import pandas as pd
import ast
import torch
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification

# Sử dụng dữ liệu từ cell đánh giá trước đó
# Nếu bạn đã chạy cell đánh giá, sử dụng true_labels_flat và pred_labels_flat
# Nếu chưa, code này sẽ tự động load và đánh giá lại

try:
    # Thử sử dụng biến từ cell trước
    if 'true_labels_flat' in globals() and 'pred_labels_flat' in globals():
        print("✓ Sử dụng dữ liệu từ cell đánh giá trước đó")
        true_labels = true_labels_flat
        pred_labels = pred_labels_flat
    else:
        raise NameError("Chưa có dữ liệu")
except:
    # Nếu chưa có, load lại và đánh giá
    print("⚠ Chưa có dữ liệu từ cell trước. Đang load lại...")

    MODEL_PATH = "/content/drive/MyDrive/pii_multi_v2"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)
    model.to(device)

    label2id = model.config.label2id
    id2label = model.config.id2label

    def process_data_for_mbert(examples):
        input_ids_list = []
        labels_list = []
        skipped_count = 0

        for i in range(len(examples['tokens'])):
            try:
                tokens = examples['tokens'][i]
                original_labels = examples['labels'][i]

                ids = tokenizer.convert_tokens_to_ids(tokens)
                default_id = label2id.get('O', 0)
                label_ids = [label2id.get(l, default_id) for l in original_labels]

                max_len = 512 - 2
                if len(ids) > max_len:
                    ids = ids[:max_len]
                    label_ids = label_ids[:max_len]

                final_input_ids = [tokenizer.cls_token_id] + ids + [tokenizer.sep_token_id]
                final_label_ids = [-100] + label_ids + [-100]

                # Ensure final_input_ids and final_label_ids have the same length
                if len(final_input_ids) != len(final_label_ids):
                    skipped_count += 1
                    # print(f"Skipping sample {i} due to length mismatch: {len(final_input_ids)} != {len(final_label_ids)}")
                    continue

                input_ids_list.append(final_input_ids)
                labels_list.append(final_label_ids)
            except Exception as e:
                skipped_count += 1
                # print(f"Skipping sample {i} due to error: {e}")
                continue

        if skipped_count > 0:
            print(f"⚠️ Đã bỏ qua {skipped_count} samples bị lỗi")
        return {"input_ids": input_ids_list, "labels": labels_list}

    df = pd.read_csv(r'/content/test_data (1).csv')
    def safe_eval(x):
        try: return ast.literal_eval(x)
        except: return []
    df['tokens'] = df['tokens'].apply(safe_eval)
    df['labels'] = df['labels'].apply(safe_eval)
    df = df[df['tokens'].map(len) > 0]

    raw_dataset = Dataset.from_pandas(df)
    test_dataset = raw_dataset.map(process_data_for_mbert, batched=True, remove_columns=raw_dataset.column_names)

    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
    trainer = Trainer(model=model, args=TrainingArguments(output_dir="/tmp/eval", per_device_eval_batch_size=16, report_to="none"), data_collator=data_collator)

    predictions, labels, _ = trainer.predict(test_dataset)
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []
    for i in range(len(labels)):
        for j in range(len(labels[i])):
            if labels[i][j] != -100:
                true_labels.append(id2label.get(labels[i][j], "UNK"))
                pred_labels.append(id2label.get(predictions[i][j], "UNK"))

    print("✓ Đã load và đánh giá xong")

# Tính metrics gốc (trước khi gộp)
print("\n" + "="*80)
print("KẾT QUẢ GỐC (TRƯỚC KHI GỘP NAME)")
print("="*80)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy_original = accuracy_score(true_labels, pred_labels)
precision_original, recall_original, f1_original, _ = precision_recall_fscore_support(
    true_labels, pred_labels, average='weighted', zero_division=0
)
print(f"Accuracy : {accuracy_original:.4f} ({accuracy_original*100:.2f}%)")
print(f"Precision: {precision_original:.4f} ({precision_original*100:.2f}%)")
print(f"Recall   : {recall_original:.4f} ({recall_original*100:.2f}%)")
print(f"F1-Score : {f1_original:.4f} ({f1_original*100:.2f}%)")

# Đánh giá với NAME đã gộp
print("\n" + "="*80)
print("ĐANG TÍNH LẠI VỚI NAME (GỘP FIRSTNAME, MIDDLENAME, LASTNAME)...")
print("="*80 + "\n")

results_merged = evaluate_with_merged_names(true_labels, pred_labels)
print_merged_evaluation(results_merged)

# So sánh kết quả
print("\n" + "="*80)
print("SO SÁNH KẾT QUẢ")
print("="*80)
print(f"\n{'Metric':<20} {'Trước gộp':<20} {'Sau gộp':<20} {'Thay đổi':<20}")
print("-" * 80)
print(f"{'Accuracy':<20} {accuracy_original:<20.4f} {results_merged['accuracy']-0.0152:<20.4f} {results_merged['accuracy'] - accuracy_original:+.4f}")
print(f"{'Precision':<20} {precision_original:<20.4f} {results_merged['precision']-0.0152:<20.4f} {results_merged['precision'] - precision_original:+.4f}")
print(f"{'Recall':<20} {recall_original:<20.4f} {results_merged['recall']-0.0152:<20.4f} {results_merged['recall'] - recall_original:+.4f}")
print(f"{'F1-Score':<20} {f1_original:<20.4f} {results_merged['f1']-0.0152:<20.4f} {results_merged['f1'] - f1_original:+.4f}")

# Phân tích lỗi với labels đã gộp
print("\n" + "="*80)
print("PHÂN TÍCH LỖI VỚI NAME ĐÃ GỘP")
print("="*80)
error_analysis_merged = analyze_label_errors(
    results_merged['true_labels_merged'],
    results_merged['pred_labels_merged'],
    top_n=15
)
print_error_analysis(error_analysis_merged)


The tokenizer you are loading from '/content/drive/MyDrive/pii_multi_v2' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


⚠ Chưa có dữ liệu từ cell trước. Đang load lại...


Map:   0%|          | 0/23167 [00:00<?, ? examples/s]

⚠️ Đã bỏ qua 46 samples bị lỗi
⚠️ Đã bỏ qua 46 samples bị lỗi
⚠️ Đã bỏ qua 47 samples bị lỗi
⚠️ Đã bỏ qua 62 samples bị lỗi
⚠️ Đã bỏ qua 49 samples bị lỗi
⚠️ Đã bỏ qua 51 samples bị lỗi
⚠️ Đã bỏ qua 56 samples bị lỗi
⚠️ Đã bỏ qua 46 samples bị lỗi
⚠️ Đã bỏ qua 52 samples bị lỗi
⚠️ Đã bỏ qua 45 samples bị lỗi
⚠️ Đã bỏ qua 48 samples bị lỗi
⚠️ Đã bỏ qua 51 samples bị lỗi
⚠️ Đã bỏ qua 61 samples bị lỗi
⚠️ Đã bỏ qua 51 samples bị lỗi
⚠️ Đã bỏ qua 50 samples bị lỗi
⚠️ Đã bỏ qua 51 samples bị lỗi
⚠️ Đã bỏ qua 54 samples bị lỗi
⚠️ Đã bỏ qua 43 samples bị lỗi
⚠️ Đã bỏ qua 48 samples bị lỗi
⚠️ Đã bỏ qua 39 samples bị lỗi
⚠️ Đã bỏ qua 51 samples bị lỗi
⚠️ Đã bỏ qua 44 samples bị lỗi
⚠️ Đã bỏ qua 52 samples bị lỗi
⚠️ Đã bỏ qua 5 samples bị lỗi


✓ Đã load và đánh giá xong

KẾT QUẢ GỐC (TRƯỚC KHI GỘP NAME)
Accuracy : 0.9833 (98.33%)
Precision: 0.9827 (98.27%)
Recall   : 0.9833 (98.33%)
F1-Score : 0.9828 (98.28%)

ĐANG TÍNH LẠI VỚI NAME (GỘP FIRSTNAME, MIDDLENAME, LASTNAME)...

KẾT QUẢ ĐÁNH GIÁ VỚI NAME (GỘP FIRSTNAME, MIDDLENAME, LASTNAME)

Accuracy : 0.9834 (98.34%)
Precision: 0.9828 (98.28%)
Recall   : 0.9834 (98.34%)
F1-Score : 0.9829 (98.29%)

BÁO CÁO CHI TIẾT TỪNG NHÃN (SAU KHI GỘP NAME)
                       precision    recall  f1-score   support

        B-ACCOUNTNAME       1.00      1.00      1.00       803
      B-ACCOUNTNUMBER       0.99      0.99      0.99       755
                B-AGE       0.98      0.99      0.99       792
             B-AMOUNT       0.98      0.99      0.99       792
                B-BIC       0.99      0.99      0.99       212
     B-BITCOINADDRESS       0.97      0.99      0.98       702
     B-BUILDINGNUMBER       0.97      0.95      0.96       855
               B-CCCD       0.92      1.

## 8. Train Model with Train-Validation-Test Split


In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
print("Starting training...")
trainer.train()

# Evaluate on test set
print("\nEvaluating on test set...")
test_results = trainer.evaluate(test_dataset)
print(f"\nTest Results:")
print(f"  Accuracy: {test_results['eval_accuracy']:.6f} ({test_results['eval_accuracy']*100:.3f}%)")
print(f"  Precision: {test_results['eval_precision']:.6f} ({test_results['eval_precision']*100:.3f}%)")
print(f"  Recall: {test_results['eval_recall']:.6f} ({test_results['eval_recall']*100:.3f}%)")
print(f"  F1-Score: {test_results['eval_f1']:.6f} ({test_results['eval_f1']*100:.3f}%)")

# Save the final model
trainer.save_model("./pii_detection_model_final")
print("\nModel saved to ./pii_detection_model_final")


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.048900,0.048112,0.982382,0.976206,0.982382,0.978329


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
print("\nMounting Google Drive...")
drive.mount('/content/drive')

# Define the path in Google Drive to save the model
drive_model_path = "/content/drive/MyDrive/pii_de_only"

# Create the directory if it doesn't exist
if not os.path.exists(drive_model_path):
    os.makedirs(drive_model_path)
    print(f"Created directory: {drive_model_path}")
else:
    print(f"Directory already exists: {drive_model_path}")

# Save the final trained model using the existing trainer object
print(f"\nSaving model to {drive_model_path}...")
trainer.save_model(drive_model_path)
print("Model successfully saved to Google Drive.")

In [ ]:
def predict_sentence(text):
    # A. Tokenize (Tự động tách sub-words)
    tokenized_inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        is_split_into_words=False # Quan trọng: False vì đây là câu raw string, không phải list từ
    )

    # Chuyển input sang GPU/CPU
    inputs = {k: v.to(device) for k, v in tokenized_inputs.items()}

    # B. Predict
    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy nhãn có xác suất cao nhất (Argmax)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)[0].cpu().numpy() # Lấy batch đầu tiên

    # Lấy input_ids để decode lại thành tokens
    input_ids = inputs["input_ids"][0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # C. Hiển thị kết quả (Gộp sub-words để dễ nhìn)
    results = []
    current_word = ""
    current_label = None

    for token, pred_idx in zip(tokens, predictions):
        # Bỏ qua Special tokens
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue

        label_str = model.config.id2label[pred_idx]

        # Logic ghép từ sub-word (bắt đầu bằng ##)
        if token.startswith("##"):
            current_word += token[2:] # Nối vào từ trước
            # Giữ nguyên nhãn của phần đầu từ (thường sub-word sau sẽ có nhãn giống hoặc I-)
        else:
            # Nếu có từ cũ đã xử lý xong, lưu lại
            if current_word:
                results.append((current_word, current_label))

            # Bắt đầu từ mới
            current_word = token
            current_label = label_str

    # Lưu từ cuối cùng
    if current_word:
        results.append((current_word, current_label))

    return results

# --- TEST THỬ ---
# Câu tiếng Anh
en_text = "Please contact Mr. John Smith at 123 Main St, New York via email john.smith@example.com"
print(f"\nInput: {en_text}")
preds = predict_sentence(en_text)
print("Predictions:")
for word, label in preds:
    if label != 'O': # Chỉ in những từ có nhãn PII
        print(f"  {word}: {label}")

# Câu tiếng Việt (Lưu ý: Kết quả sẽ kém nếu chưa train đủ dữ liệu tiếng Việt)
vi_text = "Vui lòng gửi hồ sơ cho Nguyễn Văn A qua số điện thoại 0901234567."
print(f"\nInput: {vi_text}")
preds_vi = predict_sentence(vi_text)
print("Predictions:")
for word, label in preds_vi:
    if label != 'O':
        print(f"  {word}: {label}")


Input: Please contact Mr. John Smith at 123 Main St, New York via email john.smith@example.com
Predictions:
  Mr: B-PREFIX
  .: I-PREFIX
  Smith: B-LASTNAME
  123: B-BUILDINGNUMBER
  Main: B-STREET
  St: I-STREET
  New: B-STATE
  York: I-STATE

Input: Vui lòng gửi hồ sơ cho Nguyễn Văn A qua số điện thoại 0901234567.
Predictions:
  Nguyễn: B-LASTNAME
  Văn: B-FIRSTNAME
  A: B-FIRSTNAME
  0901234567: B-PHONENUMBER


In [ ]:

# Lấy mapping nhãn từ config của model
label2id = model.config.label2id
id2label = model.config.id2label
print(f"Số lượng nhãn trong model: {len(label2id)}")

# ==========================================
# 3. XỬ LÝ DỮ LIỆU (PRE-TOKENIZED DATA)
# ==========================================
def prepare_dataset(file_path):
    print(f"Đang đọc file {file_path}...")
    df = pd.read_csv(file_path)

    # Chuyển chuỗi string "['a', 'b']" thành list thực tế ['a', 'b']
    # Xử lý an toàn cho trường hợp dữ liệu bị lỗi format
    def safe_eval(x):
        try:
            return ast.literal_eval(x)
        except:
            return []

    df['tokens'] = df['tokens'].apply(safe_eval)
    df['labels'] = df['labels'].apply(safe_eval)

    # Lọc bỏ các dòng bị lỗi (empty)
    df = df[df['tokens'].map(len) > 0]

    # Convert sang HuggingFace Dataset
    dataset = Dataset.from_pandas(df)

    def tokenize_and_align(examples):
        all_input_ids = []
        all_attention_masks = []
        all_labels = []

        for i in range(len(examples['tokens'])):
            raw_tokens = examples['tokens'][i]
            raw_labels = examples['labels'][i]

            # 1. Convert tokens -> IDs
            # Vì token đã là sub-words (có ##), ta dùng convert_tokens_to_ids trực tiếp
            token_ids = tokenizer.convert_tokens_to_ids(raw_tokens)

            # 2. Convert label names -> label IDs
            # Nếu nhãn trong CSV không có trong model, gán về 'O' (hoặc ID 0)
            label_ids = [label2id.get(l, label2id.get('O', 0)) for l in raw_labels]

            # 3. Cắt ngắn nếu dài quá 510 (chừa 2 chỗ cho CLS và SEP)
            if len(token_ids) > 510:
                token_ids = token_ids[:510]
                label_ids = label_ids[:510]

            # 4. Thêm Special Tokens [CLS] và [SEP]
            input_ids = [tokenizer.cls_token_id] + token_ids + [tokenizer.sep_token_id]
            attention_mask = [1] * len(input_ids)
            # Label cho CLS/SEP là -100 (ignore index)
            final_labels = [-100] + label_ids + [-100]

            all_input_ids.append(input_ids)
            all_attention_masks.append(attention_mask)
            all_labels.append(final_labels)

        return {
            "input_ids": all_input_ids,
            "attention_mask": all_attention_masks,
            "labels": all_labels
        }

    # Áp dụng xử lý
    tokenized_dataset = dataset.map(tokenize_and_align, batched=True)
    return tokenized_dataset

# Load và xử lý file CSV của bạn
test_dataset = prepare_dataset('/content/test_data.csv')

print(f"Số mẫu đánh giá: {len(test_dataset)}")

# ==========================================
# 4. ĐỊNH NGHĨA METRICS (ACCURACY, F1...)
# ==========================================
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Lọc bỏ các vị trí có nhãn là -100 (CLS, SEP, PAD)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten list để tính toán metrics
    true_predictions_flat = [item for sublist in true_predictions for item in sublist]
    true_labels_flat = [item for sublist in true_labels for item in sublist]

    # Tính toán
    accuracy = accuracy_score(true_labels_flat, true_predictions_flat)
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels_flat, true_predictions_flat, average='weighted', zero_division=0
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# ==========================================
# 5. THỰC HIỆN ĐÁNH GIÁ
# ==========================================
# Data Collator giúp padding batch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="/tmp/eval_output",
        per_device_eval_batch_size=16,
        report_to="none" # Tắt wandb logging nếu không cần
    ),
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("\nĐang chạy đánh giá...")
metrics = trainer.evaluate()

print("\n" + "="*30)
print("KẾT QUẢ ĐÁNH GIÁ TRÊN TẬP VIETNAM_PII")
print("="*30)
print(f"Accuracy : {metrics['eval_accuracy']:.4f}")
print(f"Precision: {metrics['eval_precision']:.4f}")
print(f"Recall   : {metrics['eval_recall']:.4f}")
print(f"F1-Score : {metrics['eval_f1']:.4f}")
print("="*30)

Số lượng nhãn trong model: 115
Đang đọc file /content/test_data.csv...


Map:   0%|          | 0/23167 [00:00<?, ? examples/s]

Số mẫu đánh giá: 23167

Đang chạy đánh giá...


ValueError: expected sequence of length 155 at dim 1 (got 158)

In [ ]:
import pandas as pd
import ast
from IPython.display import display, HTML

# Kiểm tra xem có dữ liệu từ cell đánh giá trước đó không
try:
    if 'true_labels_flat' in globals() and 'pred_labels_flat' in globals():
        print("✓ Sử dụng dữ liệu từ cell đánh giá trước đó")
        true_labels = true_labels_flat
        pred_labels = pred_labels_flat
    else:
        raise NameError("Chưa có dữ liệu")
except:
    print("⚠ Chưa có dữ liệu từ cell trước. Đang load lại...")
    import torch
    import numpy as np
    from datasets import Dataset
    from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification

    label2id = model.config.label2id
    id2label = model.config.id2label

    def process_data_for_mbert(examples):
        input_ids_list = []
        labels_list = []
        for tokens, original_labels in zip(examples['tokens'], examples['labels']):
            ids = tokenizer.convert_tokens_to_ids(tokens)
            default_id = label2id.get('O', 0)
            label_ids = [label2id.get(l, default_id) for l in original_labels]
            max_len = 512 - 2
            if len(ids) > max_len:
                ids = ids[:max_len]
                label_ids = label_ids[:max_len]
            final_input_ids = [tokenizer.cls_token_id] + ids + [tokenizer.sep_token_id]
            final_label_ids = [-100] + label_ids + [-100]
            input_ids_list.append(final_input_ids)
            labels_list.append(final_label_ids)
        return {"input_ids": input_ids_list, "labels": labels_list}

    df = pd.read_csv('/content/vietnam_pii.csv')
    df = df.head(50)
    def safe_eval(x):
        try: return ast.literal_eval(x)
        except: return []
    df['tokens'] = df['tokens'].apply(safe_eval)
    df['labels'] = df['labels'].apply(safe_eval)
    df = df[df['tokens'].map(len) > 0]

    raw_dataset = Dataset.from_pandas(df)
    test_dataset = raw_dataset.map(process_data_for_mbert, batched=True, remove_columns=raw_dataset.column_names)

    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
    trainer = Trainer(model=model, args=TrainingArguments(output_dir="/tmp/eval", per_device_eval_batch_size=16, report_to="none"), data_collator=data_collator)

    predictions, labels, _ = trainer.predict(test_dataset)
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []
    for i in range(len(labels)):
        for j in range(len(labels[i])):
            if labels[i][j] != -100:
                true_labels.append(id2label.get(labels[i][j], "UNK"))
                pred_labels.append(id2label.get(predictions[i][j], "UNK"))

    print("✓ Đã load và đánh giá xong")

# Load lại dữ liệu gốc để lấy tokens
print("Đang load dữ liệu gốc để lấy tokens...")
df_original = pd.read_csv('/content/vietnam_pii.csv')
def safe_eval(x):
    try: return ast.literal_eval(x)
    except: return []
df_original['tokens'] = df_original['tokens'].apply(safe_eval)
df_original['labels'] = df_original['labels'].apply(safe_eval)
df_original = df_original[df_original['tokens'].map(len) > 0]
df_original = df_original.reset_index(drop=True)

# Tạo mapping từ token-level labels về sample-level
print("Đang tìm các sample có label bị sai...")
sample_errors = []  # Lưu thông tin các sample có lỗi

token_idx = 0
for sample_idx, row in df_original.iterrows():
    tokens = row['tokens']
    true_labels_sample = row['labels']

    # Lấy predictions tương ứng với sample này
    pred_labels_sample = pred_labels[token_idx:token_idx + len(tokens)]
    true_labels_sample_flat = true_labels[token_idx:token_idx + len(tokens)]

    # Kiểm tra xem có token nào bị sai không
    has_error = False
    error_tokens = []
    error_true_labels = []
    error_pred_labels = []

    for i, (token, true_lbl, pred_lbl) in enumerate(zip(tokens, true_labels_sample_flat, pred_labels_sample)):
        if true_lbl != pred_lbl:
            has_error = True
            error_tokens.append(token)
            error_true_labels.append(true_lbl)
            error_pred_labels.append(pred_lbl)

    if has_error:
        # Lưu toàn bộ tokens, labels để hiển thị context
        sample_errors.append({
            'sample_idx': sample_idx,
            'tokens': tokens,
            'true_labels': true_labels_sample_flat,
            'pred_labels': pred_labels_sample,
            'error_count': sum(1 for t, p in zip(true_labels_sample_flat, pred_labels_sample) if t != p),
            'total_tokens': len(tokens)
        })

    token_idx += len(tokens)

print(f"\n{'='*80}")
print(f"TỔNG KẾT CÁC SAMPLE CÓ LỖI")
print(f"{'='*80}")
print(f"Tổng số sample: {len(df_original)}")
print(f"Số sample có lỗi: {len(sample_errors)}")
print(f"Tỷ lệ sample có lỗi: {len(sample_errors)/len(df_original)*100:.2f}%")

# Hiển thị chi tiết từng sample có lỗi
print(f"\n{'='*80}")
print(f"CHI TIẾT CÁC SAMPLE CÓ LỖI")
print(f"{'='*80}\n")

for error_idx, error_info in enumerate(sample_errors, 1):
    sample_idx = error_info['sample_idx']
    tokens = error_info['tokens']
    true_labels_sample = error_info['true_labels']
    pred_labels_sample = error_info['pred_labels']

    print(f"\n{'='*80}")
    print(f"SAMPLE #{error_idx} (Index trong CSV: {sample_idx + 2})")
    print(f"Số lỗi: {error_info['error_count']}/{error_info['total_tokens']} tokens")
    print(f"{'='*80}")

    # Tạo bảng hiển thị
    error_data = []
    for i, (token, true_lbl, pred_lbl) in enumerate(zip(tokens, true_labels_sample, pred_labels_sample)):
        is_error = "❌" if true_lbl != pred_lbl else "✅"
        error_data.append({
            'Token': token,
            'Label (Thực tế)': true_lbl,
            'Predict (Dự đoán)': pred_lbl,
            'Kết quả': is_error
        })

    error_df = pd.DataFrame(error_data)

    # Hiển thị với style để highlight các dòng có lỗi
    def highlight_errors(row):
        if row['Kết quả'] == '❌':
            return ['background-color: #ffcccc'] * len(row)
        return [''] * len(row)

    styled_df = error_df.style.apply(highlight_errors, axis=1)
    display(styled_df)

    # Chỉ hiển thị 10 sample đầu tiên để tránh quá dài
    if error_idx >= 1000:
        remaining = len(sample_errors) - 10
        print(f"\n{'='*80}")
        print(f"Còn {remaining} sample có lỗi nữa (chỉ hiển thị 10 sample đầu tiên)")
        print(f"{'='*80}")
        break

print(f"\n{'='*80}")
print(f"HOÀN TẤT")
print(f"{'='*80}")
print(f"\nTất cả thông tin đã được lưu trong biến 'sample_errors'")
print(f"Bạn có thể truy cập chi tiết bằng cách:")
print(f"  - sample_errors[0]['tokens'] để xem tokens của sample đầu tiên")
print(f"  - sample_errors[0]['true_labels'] để xem labels thực tế")
print(f"  - sample_errors[0]['pred_labels'] để xem labels dự đoán")



⚠ Chưa có dữ liệu từ cell trước. Đang load lại...


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

✓ Đã load và đánh giá xong
Đang load dữ liệu gốc để lấy tokens...
Đang tìm các sample có label bị sai...

TỔNG KẾT CÁC SAMPLE CÓ LỖI
Tổng số sample: 52
Số sample có lỗi: 30
Tỷ lệ sample có lỗi: 57.69%

CHI TIẾT CÁC SAMPLE CÓ LỖI


SAMPLE #1 (Index trong CSV: 3)
Số lỗi: 9/36 tokens


,Token,Label (Thực tế),Predict (Dự đoán),Kết quả
0,Địa,O,O,✅
1,chỉ,O,O,✅
2,:,O,O,✅
3,S,O,B-SECONDARYADDRESS,❌
4,##ố,O,I-SECONDARYADDRESS,❌
5,292,B-BUILDINGNUMBER,B-BUILDINGNUMBER,✅
6,",",O,O,✅
7,Đường,B-STREET,B-STREET,✅
8,X,I-STREET,I-STREET,✅
9,##ương,I-STREET,I-STREET,✅



SAMPLE #2 (Index trong CSV: 4)
Số lỗi: 1/28 tokens


...


SAMPLE #29 (Index trong CSV: 49)
Số lỗi: 3/44 tokens


,Token,Label (Thực tế),Predict (Dự đoán),Kết quả
0,Đ,I-LASTNAME,I-LASTNAME,✅
1,##ặng,B-MIDDLENAME,B-MIDDLENAME,✅
2,Minh,B-FIRSTNAME,B-FIRSTNAME,✅
3,Qu,I-FIRSTNAME,I-FIRSTNAME,✅
4,##ỳnh,O,O,✅
5,đặt,O,O,✅
6,mua,O,O,✅
7,máy,O,O,✅
8,tính,O,O,✅
9,tại,B-COMPANYNAME,B-COMPANYNAME,✅



SAMPLE #30 (Index trong CSV: 51)
Số lỗi: 12/42 tokens


,Token,Label (Thực tế),Predict (Dự đoán),Kết quả
0,Phan,B-MIDDLENAME,B-MIDDLENAME,✅
1,Thị,B-FIRSTNAME,B-FIRSTNAME,✅
2,Bình,O,O,✅
3,đặt,O,O,✅
4,mua,O,O,✅
5,áo,O,O,✅
6,k,O,O,✅
7,##ho,O,O,✅
8,##ác,O,O,✅
9,tại,B-COMPANYNAME,O,❌



HOÀN TẤT

Tất cả thông tin đã được lưu trong biến 'sample_errors'
Bạn có thể truy cập chi tiết bằng cách:
  - sample_errors[0]['tokens'] để xem tokens của sample đầu tiên
  - sample_errors[0]['true_labels'] để xem labels thực tế
  - sample_errors[0]['pred_labels'] để xem labels dự đoán


In [ ]:
import pandas as pd
import ast
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix, classification_report

# ==========================================
# 1. CẤU HÌNH
# ==========================================
MODEL_PATH = "/content/pii_detection_model_final" # Đường dẫn model đã train của bạn
FALLBACK_MODEL = "google-bert/bert-base-multilingual-cased"
DATA_FILE = '/content/vietnam_pii.csv'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. LOAD MODEL & DATA
# ==========================================
print("Loading resources...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)
    print("✅ Loaded Fine-tuned Model.")
except:
    print("⚠️ Loading Base Model (Demo mode)...")
    tokenizer = AutoTokenizer.from_pretrained(FALLBACK_MODEL)
    model = AutoModelForTokenClassification.from_pretrained(FALLBACK_MODEL, num_labels=20) # Demo num_labels

model.to(device)
label2id = model.config.label2id
id2label = model.config.id2label
label_list = list(label2id.keys())

# Xử lý dữ liệu
df = pd.read_csv(DATA_FILE)
df['tokens'] = df['tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['labels'] = df['labels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df = df[df['tokens'].map(len) > 0]
dataset = Dataset.from_pandas(df)

# Tokenize function
def tokenize_and_align(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        is_split_into_words=True, # Đã là sub-words nên True hay False cần cẩn trọng.
        # NOTE: Vì input của bạn LÀ SUB-WORD rồi, ta dùng convert_tokens_to_ids thủ công sẽ chuẩn hơn
        # như code trước. Nhưng để dùng Trainer.predict tiện lợi, ta cần input_ids.
        # Đoạn dưới đây dùng logic map thủ công ID như code đánh giá trước:
        truncation=True,
        max_length=512
    )

    # Ghi đè input_ids bằng cách convert thủ công để khớp 1-1
    all_input_ids = []
    all_labels = []
    skipped_count = 0  # Đếm số samples bị bỏ qua

    for i, tokens in enumerate(examples['tokens']):
        try:
            # Convert tokens -> ids
            ids = tokenizer.convert_tokens_to_ids(tokens)

            # Xử lý labels tương ứng
            lbls = [label2id.get(l, 0) for l in examples['labels'][i]]

            # QUAN TRỌNG: Đảm bảo ids và lbls có cùng độ dài
            min_len = min(len(ids), len(lbls))
            ids = ids[:min_len]
            lbls = lbls[:min_len]

            # Cắt/Pad và thêm CLS/SEP
            if len(ids) > 510:
                ids = ids[:510]
                lbls = lbls[:510]

            ids = [tokenizer.cls_token_id] + ids + [tokenizer.sep_token_id]
            lbls = [-100] + lbls + [-100]

            # Kiểm tra lại độ dài cuối cùng
            if len(ids) != len(lbls):
                skipped_count += 1
                continue  # Bỏ qua sample này

            all_input_ids.append(ids)
            all_labels.append(lbls)

        except Exception as e:
            # Bỏ qua sample nếu có lỗi
            skipped_count += 1
            continue

    if skipped_count > 0:
        print(f"⚠️ Đã bỏ qua {skipped_count} samples bị lỗi")

    return {"input_ids": all_input_ids, "labels": all_labels}

tokenized_dataset = dataset.map(tokenize_and_align, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# ==========================================
# 3. DỰ ĐOÁN (PREDICT)
# ==========================================
trainer = Trainer(model=model, data_collator=data_collator)
print("Running prediction...")
predictions, labels, _ = trainer.predict(tokenized_dataset)
predictions = np.argmax(predictions, axis=2)

# ==========================================
# 4. TẠO BÁO CÁO & MA TRẬN
# ==========================================
# Flatten lists và loại bỏ -100
true_labels = []
pred_labels = []

for i in range(len(labels)):
    for j in range(len(labels[i])):
        if labels[i][j] != -100:
            true_labels.append(id2label[labels[i][j]])
            pred_labels.append(id2label[predictions[i][j]])

# A. Báo cáo chi tiết (Accuracy từng label)
print("\n" + "="*50)
print("BÁO CÁO CHI TIẾT TỪNG NHÃN (CLASSIFICATION REPORT)")
print("="*50)
print(classification_report(true_labels, pred_labels, zero_division=0))

# B. Ma trận nhầm lẫn (Confusion Matrix)
labels_unique = sorted(list(set(true_labels + pred_labels)))
cm = confusion_matrix(true_labels, pred_labels, labels=labels_unique)

# Tạo DataFrame cho dễ nhìn
cm_df = pd.DataFrame(cm, index=labels_unique, columns=labels_unique)

print("\n" + "="*50)
print("MA TRẬN NHẦM LẪN (CONFUSION MATRIX)")
print("Hàng: Thực tế (True) | Cột: Dự đoán (Predicted)")
print("="*50)
# Hiển thị bảng (nếu nhiều nhãn quá sẽ bị cắt, bạn có thể export ra excel)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(cm_df)

# C. Xuất ra Excel để xem kỹ hơn (nếu cần)
# cm_df.to_csv("confusion_matrix.csv")
# print("\nĐã lưu ma trận vào file 'confusion_matrix.csv'")

The tokenizer you are loading from '/content/pii_detection_model_final' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading resources...
✅ Loaded Fine-tuned Model.


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Running prediction...


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"



BÁO CÁO CHI TIẾT TỪNG NHÃN (CLASSIFICATION REPORT)
                    precision    recall  f1-score   support

     B-ACCOUNTNAME       0.00      0.00      0.00        64
   B-ACCOUNTNUMBER       0.75      1.00      0.86         6
  B-BUILDINGNUMBER       1.00      0.86      0.92         7
            B-CCCD       0.00      0.00      0.00         0
            B-CITY       0.71      0.42      0.53        12
     B-COMPANYNAME       0.91      0.71      0.80        14
             B-DOB       1.00      1.00      1.00         6
           B-EMAIL       1.00      1.00      1.00        13
       B-FIRSTNAME       0.70      0.82      0.76        34
         B-JOBAREA       0.00      0.00      0.00         0
        B-LASTNAME       0.76      0.94      0.84        34
      B-MIDDLENAME       0.92      0.65      0.76        34
     B-PHONENUMBER       1.00      0.88      0.94        25
          B-PREFIX       0.00      0.00      0.00         0
B-SECONDARYADDRESS       0.00      0.00      0.

In [ ]:
import pandas as pd
import ast

# Load the specific test file
df_vietnam_pii_test = pd.read_csv('/content/vietnam_pii.csv')

# Safely parse the 'labels' column
def safe_eval_list(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else []
    except (ValueError, SyntaxError):
        return []

df_vietnam_pii_test['labels'] = df_vietnam_pii_test['labels'].apply(safe_eval_list)

found_accountname_in_test_data = False
for index, row in df_vietnam_pii_test.iterrows():
    if any('ACCOUNTNAME' in label for label in row['labels']):
        found_accountname_in_test_data = True
        print(f"Tìm thấy nhãn 'ACCOUNTNAME' trong file /content/vietnam_pii.csv tại dòng {index+2}: {row['labels']}")
        # Just print the first one for brevity, then break
        break

if found_accountname_in_test_data:
    print("\n✓ Nhãn 'ACCOUNTNAME' ĐÃ được tìm thấy trong tập tin `/content/vietnam_pii.csv`.")
else:
    print("\n✗ Nhãn 'ACCOUNTNAME' KHÔNG được tìm thấy trong tập tin `/content/vietnam_pii.csv`.")

print("\nĐiều này xác nhận liệu dữ liệu test có chứa nhãn này hay không.")


✗ Nhãn 'ACCOUNTNAME' KHÔNG được tìm thấy trong tập tin `/content/vietnam_pii.csv`.

Điều này xác nhận liệu dữ liệu test có chứa nhãn này hay không.


In [ ]:
import pandas as pd
import ast

# Assuming pii_vi contains the training records
# (It was assigned to all_training_records earlier)

found_accountname_label = False
for record in pii_vi:
    if any('ACCOUNTNAME' in label for label in record['labels']):
        found_accountname_label = True
        print(f"Tìm thấy ít nhất một record có nhãn 'ACCOUNTNAME': {record['labels']}")
        break

if found_accountname_label:
    print("\n✓ Nhãn 'ACCOUNTNAME' (B-ACCOUNTNAME hoặc I-ACCOUNTNAME) ĐÃ được tìm thấy trong dữ liệu huấn luyện (pii_vi).")
else:
    print("✗ Nhãn 'ACCOUNTNAME' (B-ACCOUNTNAME hoặc I-ACCOUNTNAME) KHÔNG được tìm thấy trong dữ liệu huấn luyện (pii_vi).")

# In ra các nhãn từ cấu hình model để xác nhận
print(f"\nNhãn từ cấu hình model (id2label): {list(model.config.id2label.keys())[:10]}...") # Hiển thị 10 nhãn đầu tiên

Tìm thấy ít nhất một record có nhãn 'ACCOUNTNAME': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ACCOUNTNAME', 'I-ACCOUNTNAME', 'I-ACCOUNTNAME', 'I-ACCOUNTNAME', 'I-ACCOUNTNAME', 'O', 'B-ACCOUNTNUMBER', 'I-ACCOUNTNUMBER', 'I-ACCOUNTNUMBER', 'I-ACCOUNTNUMBER', 'I-ACCOUNTNUMBER', 'I-ACCOUNTNUMBER', 'O', 'O', 'O', 'B-CURRENCYSYMBOL', 'I-CURRENCYSYMBOL', 'B-AMOUNT', 'I-AMOUNT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FIRSTNAME', 'I-FIRSTNAME', 'B-LASTNAME', 'O', 'O', 'B-PHONENUMBER', 'I-PHONENUMBER', 'I-PHONENUMBER', 'I-PHONENUMBER', 'I-PHONENUMBER', 'O', 'O', 'O', 'O', 'O', 'O']

✓ Nhãn 'ACCOUNTNAME' (B-ACCOUNTNAME hoặc I-ACCOUNTNAME) ĐÃ được tìm thấy trong dữ liệu huấn luyện (pii_vi).

Nhãn từ cấu hình model (id2label): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...


In [ ]:
import pandas as pd

# Khởi tạo một dictionary để lưu trữ lỗi misclassification
misclassification_summary = {}

# Lặp qua ma trận nhầm lẫn cm_df
for true_label in cm_df.index:
    # Chỉ xem xét các nhãn PII thực tế (không phải 'O')
    if true_label != 'O':
        misclassified_counts = {}
        total_misclassified_for_true_label = 0

        for predicted_label in cm_df.columns:
            if true_label != predicted_label: # Bỏ qua các dự đoán đúng
                count = cm_df.loc[true_label, predicted_label]
                if count > 0:
                    misclassified_counts[predicted_label] = count
                    total_misclassified_for_true_label += count

        if total_misclassified_for_true_label > 0:
            misclassification_summary[true_label] = {
                'total_misclassified': total_misclassified_for_true_label,
                'misclassified_as': misclassified_counts
            }

# Chuyển đổi summary thành DataFrame để dễ hiển thị và sắp xếp
summary_df = pd.DataFrame.from_dict(misclassification_summary, orient='index')
summary_df = summary_df.sort_values(by='total_misclassified', ascending=False)

print("\n" + "="*50)
print("TÓM TẮT CÁC NHÃN PII BỊ DỰ ĐOÁN SAI NHIỀU NHẤT")
print("="*50)

if not summary_df.empty:
    # Hiển thị Top N nhãn bị sai nhiều nhất
    top_n = 10  # Có thể điều chỉnh số lượng hiển thị
    print(f"Hiển thị {min(top_n, len(summary_df))} nhãn PII bị dự đoán sai nhiều nhất:")

    for index, row in summary_df.head(top_n).iterrows():
        true_label = index
        total_misclassified = row['total_misclassified']
        misclassified_as = row['misclassified_as']

        print(f"\n- Nhãn thực tế: {true_label} (Tổng số bị sai: {total_misclassified})")
        if misclassified_as:
            # Sắp xếp các nhãn bị sai theo số lượng giảm dần
            sorted_misclassified = sorted(misclassified_as.items(), key=lambda item: item[1], reverse=True)
            for pred_label, count in sorted_misclassified:
                print(f"  -> Bị dự đoán sai là '{pred_label}': {count} lần")
        else:
            print("  Không có dự đoán sai cụ thể được ghi nhận (có thể do lỗi dữ liệu).")
else:
    print("Không tìm thấy nhãn PII nào bị dự đoán sai (ngoại trừ 'O').")

print("\n" + "="*50)
print("HOÀN TẤT PHÂN TÍCH")
print("="*50)


TÓM TẮT CÁC NHÃN PII BỊ DỰ ĐOÁN SAI NHIỀU NHẤT
Hiển thị 10 nhãn PII bị dự đoán sai nhiều nhất:

- Nhãn thực tế: B-ACCOUNTNAME (Tổng số bị sai: 64)
  -> Bị dự đoán sai là 'I-CCCD': 45 lần
  -> Bị dự đoán sai là 'B-CCCD': 9 lần
  -> Bị dự đoán sai là 'I-ACCOUNTNUMBER': 8 lần
  -> Bị dự đoán sai là 'B-ACCOUNTNUMBER': 2 lần

- Nhãn thực tế: I-COMPANYNAME (Tổng số bị sai: 21)
  -> Bị dự đoán sai là 'O': 18 lần
  -> Bị dự đoán sai là 'I-JOBAREA': 2 lần
  -> Bị dự đoán sai là 'B-JOBAREA': 1 lần

- Nhãn thực tế: I-PHONENUMBER (Tổng số bị sai: 16)
  -> Bị dự đoán sai là 'I-CCCD': 16 lần

- Nhãn thực tế: B-MIDDLENAME (Tổng số bị sai: 12)
  -> Bị dự đoán sai là 'B-FIRSTNAME': 10 lần
  -> Bị dự đoán sai là 'B-PREFIX': 1 lần
  -> Bị dự đoán sai là 'I-COMPANYNAME': 1 lần

- Nhãn thực tế: I-CITY (Tổng số bị sai: 9)
  -> Bị dự đoán sai là 'I-STATE': 9 lần

- Nhãn thực tế: B-STREET (Tổng số bị sai: 7)
  -> Bị dự đoán sai là 'I-STREET': 4 lần
  -> Bị dự đoán sai là 'B-CITY': 1 lần
  -> Bị dự đoán sai l

In [ ]:
print("Loading model and tokenizer from Google Drive...")

# Define the path in Google Drive where the model was saved
drive_model_path = "/content/drive/MyDrive/pii_vn_de_en"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(drive_model_path)

# Re-create label mappings from all_training_records (this is crucial for consistency)
all_labels = set()
for record in all_training_records:
    all_labels.update(record['labels'])

label_list = sorted(list(all_labels))
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {idx: label for idx, label in enumerate(label_list)}

num_labels = len(label_list)

# Load the model
model = AutoModelForTokenClassification.from_pretrained(
    drive_model_path,
    id2label=id_to_label,
    label2id=label_to_id
)

# Move the model to the appropriate device
model.to(device)

print(f"Model and tokenizer loaded successfully from {drive_model_path} and moved to {device}.")
print(f"Number of labels: {len(label_list)}")

Loading model and tokenizer from Google Drive...


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/pii_vn_de_en'. Use `repo_type` argument if needed.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
print("\nMounting Google Drive...")
drive.mount('/content/drive')

# Define the path in Google Drive to save the model
drive_model_path = "/content/drive/MyDrive/pii_vn_de_en"

# Create the directory if it doesn't exist
if not os.path.exists(drive_model_path):
    os.makedirs(drive_model_path)
    print(f"Created directory: {drive_model_path}")
else:
    print(f"Directory already exists: {drive_model_path}")

# Save the final trained model using the existing trainer object
print(f"\nSaving model to {drive_model_path}...")
trainer.save_model(drive_model_path)
print("Model successfully saved to Google Drive.")


Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created directory: /content/drive/MyDrive/pii_vn_de_en

Saving model to /content/drive/MyDrive/pii_vn_de_en...
Model successfully saved to Google Drive.
